In [ ]:
# Conectarnos al SQL Server con Python, para conectar a SQL Server, 
# necesitamos 2 librerías: pyodbc + SQLAlchemy
# >pip install pyodbc  (escribir en el CMD)

import pyodbc 
import pandas as pd

# Necesito crear una cadena de conexión, tienen diferentes parámetros 
server = r"M027619\SQLEXPRESS"   # Nombre del servidor (En el SSMS, arriba aparece el nombre (boton dch sobre él > Properties))
database = "Neptuno"             # Nombre de la base de datos (que hemos descargado del github del curso - Neptuno.2003 II.mdb )
trusted = "yes"                  # Mensaje que aparecía en SSMS - "Trust Server Certificate"

# Vamos a montar la cadena de conexión con f.strings
conn = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'               # El driver instalado - para saber cuales están instalados: pyodbc.drivers() 
    f'SERVER={server};'
    f'DATABASE={database};'
    f'TRUSTED_CONNECTION={trusted};'
)
conexion = pyodbc.connect(conn)
print("Conectado con exito")

conexion = pyodbc.connect(conn)
# Esto todo es lo que se usa si nos conectamos al SQL Server, si es una base de datos en otro servidor
# puede variar el codigo, que no vaya trusted o algo 

#Voy a cargar un DataFrame con los pedidos 
pedidos = pd.read_sql("SELECT * FROM Pedidos", conexion)
print(pedidos)

Sentencias sql con SELECT
SELECT * FROM pedidos = dame todos los campos de la tabla pedidos 
SELECT FechaPedido FROM Pedidos = me da un solo campo
SELECT FechaPedido, Destinatario FROM Pedidos = dos campos de la tabla Pedidos
SELECT FechaPedido as fecha FROM Pedidos = darle un alias al campo 
SELECT * FROM Pedidos order by Destinatario =  ordenar por un campo
SELECT * FROM Pedidos order by Destinatario, PaisDestinatario =  ordenar por un campo, ASC o DESC
SELECT * FROM Pedidos WHERE PaísDestinatario = 'Alemania'
"SELECT * FROM Pedidos WHERE PaísDestinatario = 'Alemania'" En el marco de python
SELECT * FROM Pedidos WHERE PaísDestinatario = 'Alemania'ORDER BY CiudadDestinatario
SELECT * FROM Pedidos WHERE PaísDestinatario = 'Alemania' and FechaPedido > '25/01/2025'
SELECT * FROM Pedidos WHERE PaísDestinatario = 'Alemania' or PaisDestinatario = 'Argentina'
SELECT top 30 FROM Pedidos = Me mostrara los 30 primeros (en funcion del orden)
SELECT * FROM [Detalles de pedidos] = Si hay espacios en blanco en la tabla, entre corchetes 
SELECT PrecioUnidad, Cantidad, (PrecioUnidad * Cantidad) as Importe FROM [Detalles de Pedidos]



In [ ]:
# PRACTICA 32.1:
# Realizar un codigo que se conecte a nuestra base de datos neptuno en sql server 
# Cargaremos en un dataframe con una SQL los pedidos que sea de Francia y en otro dataframe los Detalles de Pedidos 
# calculando el Importe (PrecioUnidad * Cantidad), ver en un texto la suma de importe total 

import pyodbc 
import pandas as pd

server = r"M027619\SQLEXPRESS"   
database = "Neptuno"            
trusted = "yes"                  


conn = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'             
    f'SERVER={server};'
    f'DATABASE={database};'
    f'TRUSTED_CONNECTION={trusted};'
)
conexion = pyodbc.connect(conn)
print("Conectado con exito")

conexion = pyodbc.connect(conn)

pedidos = pd.read_sql("SELECT * FROM Pedidos WHERE PaísDestinatario = 'Francia'", conexion)
detalles_pedidos = pd.read_sql("SELECT PrecioUnidad, Cantidad,IdPedido, (PrecioUnidad*Cantidad) as Importe FROM [Detalles de pedidos];",conexion)
importe_total = pd.merge(pedidos,detalles_pedidos, on="IdPedido", how="inner")
resultado = round(importe_total.Importe.sum(),2)
print("el resultado es", resultado)



In [ ]:
# PRACTICA 32.2
# Cargaremos en un dataframe los 50 primeros registros de Pedidos que sean de Finlandia o Suiza y ver en un texto el nro de pedidos totales 
registros_pedidos = pd.read_sql("SELECT TOP 50 * FROM Pedidos WHERE PaísDestinatario = 'Finlandia' or PaísDestinatario = 'Suiza'", conexion)

suma_pedidos = registros_pedidos.IdPedido.count()

print("el número de pedidos es", suma_pedidos)

In [ ]:
# Hacer una funcion para no tener que escribir el codigo todas las veces
def conectar_SQL ():
    import pyodbc 
    import pandas as pd

    server = r"M027619\SQLEXPRESS"   
    database = "Neptuno"            
    trusted = "yes"                  


    conn = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'             
    f'SERVER={server};'
    f'DATABASE={database};'
    f'TRUSTED_CONNECTION={trusted};'
    )
    conexion = pyodbc.connect(conn)

print("Conectado con exito")

conexion = pyodbc.connect(conn)


In [ ]:
#PRACTICA 32.3
# Realizar un codigo que se conecte a nuestra BBDD neptuno en sql sever 
# cargaremos las tablas necesarias para tenet en un dataframe de pandas 
# la suma del importe por nombre de categoria

conectar_SQL
conexion = pyodbc.connect(conn)

importe_categoria = pd.read_sql("SELECT dbo.Categorías.NombreCategoría," \
"dbo.[Detalles de pedidos].PrecioUnidad * dbo.[Detalles de pedidos].Cantidad AS Importe " \
"FROM dbo.Pedidos INNER JOIN dbo.[Detalles de pedidos] ON dbo.Pedidos.IdPedido = " \
"dbo.[Detalles de pedidos].IdPedido INNER JOIN dbo.Productos ON dbo.[Detalles de pedidos].IdProducto = " \
"dbo.Productos.IdProducto INNER JOIN dbo.Categorías ON dbo.Productos.IdCategoría = dbo.Categorías.IdCategoría",conexion)
importe_categoria.groupby(['NombreCategoría']).Importe.sum()

In [ ]:
consulta = "SELECT * FROM Pedidos"
pedidos = pd.read_sql(consulta,conexion)

# Vamos a trabajar con iloc (significa Integer location), cuidado warning la primera fila 
# es indice 0, la primera columna

#pedidos.iloc[indice_fila, indice_columna]
print(pedidos.iloc[0,0])


In [ ]:
# para ver el registro de las filas que contienen el nro de indice del 20 al 29
pedidos.iloc[20:30,]

In [ ]:
# Para ver contenido de las columnas
pedidos.info()

In [ ]:
# los registros con las columnas que contienen el nro de indice del 20 al 29, 10 elementos
# pero solo 3 columnas 
pedidos.iloc[20:30,[2,4,8]]

In [ ]:
pedidos.iloc[20:30,[7,2,4]]

In [ ]:
# Con iloc puedo tener un clon de todos los registros pero solo 3 columnas
pedidos3columnas = pedidos.iloc[:,[7,2,4]]

In [ ]:
# Voy a recorrer el dataframe para leer o escribir valores
pedidos.iloc[0,0] = "Hola"
pedidos

In [ ]:
# Como recorrel el dataframe, con un bucle, el nro de filas lo averiguaremos con len
filas = len(pedidos)

# Como averiguar el nro de columnas
len(pedidos.columns)

for i in range (0,filas):
    print("nombre",pedidos.iloc[i,1])

In [87]:
# Cargar en un dataframe una sentencia sql del servidor que contenga 
# los datos de detalles de pedidos, productos y categoria.
# mediante un bucle for, recorrer el dataframe y sustituir el nombre de la categoria "beverage" por "bedidas" y el nombre seafood por "pescado"
# Guardar el dataframe en un libro de excel, pero solo las filas de la 100 a la 150
detalles ="SELECT * FROM dbo.Categorías INNER JOIN dbo.Productos ON dbo.Categorías.IdCategoría = " \
"dbo.Productos.IdCategoría INNER JOIN dbo.[Detalles de pedidos] ON dbo.Productos.IdProducto = " \
"dbo.[Detalles de pedidos].IdProducto"
tablas = pd.read_sql(detalles,conexion)

#tablas.info()


for i in range (0,len(tablas)):
    if tablas.iloc[i,2]=="Bebidas":
        tablas.iloc[i,2] ="Beverages"
    elif tablas.iloc[i,2]=="Pescado/Marisco":
        tablas.iloc[i,2] ="Seafood"

tablas = tablas.iloc[100:150,]

tablas.to_excel(r"C:\temp\practica33.xlsx",sheet_name="Datos")

print ("listo")

C:\Users\formacio\AppData\Local\Temp\ipykernel_5632\1355626245.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tablas = pd.read_sql(detalles,conexion)


listo


In [ ]:
filas2 = len(detalles)

for i in range (0,filas2):
    print("Bebidas",tablas.iloc[i,1])